In [1]:
%pip install torchinfo

Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchinfo import summary

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [4]:
# download file to tmp/data.txt
!wget -O tmp/data.txt https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-09-28 21:07:02--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8002::154, 2606:50c0:8003::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: 'tmp/data.txt'

     0K .......... .......... .......... .......... ..........  4%  248K 4s
    50K .......... .......... .......... .......... ..........  9%  464K 3s
   100K .......... .......... .......... .......... .......... 13%  605K 2s
   150K .......... .......... .......... .......... .......... 18%  689K 2s
   200K .......... .......... .......... .......... .......... 22%  610K 2s
   250K .......... .......... .......... .......... .......... 27%  780K 2s
   300K .......... .......... .......... .......... .......... 32%  574K 1s
  

In [5]:
with open('tmp/data.txt', 'r') as f:
    text = f.read()
print('text length:', len(text))

text length: 1115394


In [6]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [7]:
chars = ["[PAD]", *sorted(list(set(text)))]
vocab_size = len(chars)
print("".join(chars))
print("vocab size:", vocab_size)

[PAD]
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab size: 66


In [8]:
# Create mapping from character to index and vice versa
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[47, 48, 48, 2, 59, 47, 44, 57, 44]
hii there


In [9]:
# store in tensor
data = torch.tensor(encode(text), dtype=torch.int64, device=device)
print(data.shape, data.dtype)
print(data[:100]) # the 100 characters we looked at earier will to the GPT look like this

torch.Size([1115394]) torch.int64
tensor([19, 48, 57, 58, 59,  2, 16, 48, 59, 48, 65, 44, 53, 11,  1, 15, 44, 45,
        54, 57, 44,  2, 62, 44,  2, 55, 57, 54, 42, 44, 44, 43,  2, 40, 53, 64,
         2, 45, 60, 57, 59, 47, 44, 57,  7,  2, 47, 44, 40, 57,  2, 52, 44,  2,
        58, 55, 44, 40, 50,  9,  1,  1, 14, 51, 51, 11,  1, 32, 55, 44, 40, 50,
         7,  2, 58, 55, 44, 40, 50,  9,  1,  1, 19, 48, 57, 58, 59,  2, 16, 48,
        59, 48, 65, 44, 53, 11,  1, 38, 54, 60], device='cuda:0')


In [10]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [11]:
block_size = 100
train_data[:block_size+1]

tensor([19, 48, 57, 58, 59,  2, 16, 48, 59, 48, 65, 44, 53, 11,  1, 15, 44, 45,
        54, 57, 44,  2, 62, 44,  2, 55, 57, 54, 42, 44, 44, 43,  2, 40, 53, 64,
         2, 45, 60, 57, 59, 47, 44, 57,  7,  2, 47, 44, 40, 57,  2, 52, 44,  2,
        58, 55, 44, 40, 50,  9,  1,  1, 14, 51, 51, 11,  1, 32, 55, 44, 40, 50,
         7,  2, 58, 55, 44, 40, 50,  9,  1,  1, 19, 48, 57, 58, 59,  2, 16, 48,
        59, 48, 65, 44, 53, 11,  1, 38, 54, 60,  2], device='cuda:0')

In [12]:
a = train_data[:block_size]
y = train_data[1:block_size+1]

In [13]:
torch.manual_seed(1337)
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 10 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape, xb.dtype, xb.device)
print('targets:')
print(yb.shape, yb.dtype, yb.device)

inputs:
torch.Size([32, 10]) torch.int64 cuda:0
targets:
torch.Size([32, 10]) torch.int64 cuda:0


In [109]:
class NGramLanguageModel(nn.Module):
    def __init__(self, vocab_size, n):
        super().__init__()
        super().to(device)
        self.n = n
        assert n >= 3, "n should be at least 3"
        embedding_size = int(vocab_size / 1.5 + 10)
        intermediate_size = vocab_size + 10 * n
        self.token_embedding = nn.Embedding(vocab_size, embedding_size, padding_idx=0)
        self.fc = nn.Linear(embedding_size * n, intermediate_size)
        self.dropout = nn.Dropout(0.1)
        self.relu = nn.ReLU()
        self.final = nn.Linear(intermediate_size, vocab_size)

    # Create separate function for forward calculation
    def forward(self, x, only_last=False):
        assert len(x.shape) == 2, "input shape should be (batch, time)"

        if only_last:
            # pad time dim to at least n
            x = F.pad(x, (self.n - 1, 0), value=0)
            B, T = x.shape
            x = x.view(B, 1, T)
        else:
            new_x = torch.zeros((x.shape[0], x.shape[1], self.n), dtype=torch.int64, device=device) - 69
            # -69 is just a random number so if we see it we know something went wrong
            for t in range(x.shape[1]):
                for pos in range(self.n):
                    row = x[:, max(0, t - pos) : t + 1]
                    row = F.pad(row, (self.n - row.shape[1], 0), value=0)
                    row = F.pad(row, (self.n - row.shape[1], 0), value=0)
                    new_x[:, t] = row

            x = new_x

        x = self.token_embedding(x)
        x = x.view(x.shape[0], x.shape[1], -1)
        x = self.fc(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.final(x)
        x
        return x

    # def forward(self, x, targets=None):
    #     if targets is None:
    #         # we're doing inference, so we don't have targets
    #         # if x is None, we're generating from scratch
    #         if x is None:
    #             x = batch_size

    #         # generate x batches
    #         if isinstance(x, int):
    #             x = torch.zeros(x, 1, dtype=torch.int32, device=device)
    #         return self.forward_calc(x), None

    #     else:
    #         # we're training, so we do have targets
    #         logits = self.forward_calc(x)
    #         B, T, C = logits.shape  # batch, time, channel
    #         logits_flat = logits.view(B * T, C)
    #         loss = F.cross_entropy(logits_flat, targets.view(B * T))

    #         return logits_flat, loss

    def loss(self, logits, targets):
        B, T, C = logits.shape
        logits_flat = logits.view(B * T, C)
        loss = F.cross_entropy(logits_flat, targets.view(B * T))
        return loss

    def generate(self, x, max_len_new, temperature=1.0):
        for _ in range(max_len_new):
            logits = self(x)[:, -1] / temperature
            probs = F.softmax(logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
            x = torch.cat([x, next_token], dim=1)

        return x

In [110]:
model = NGramLanguageModel(vocab_size, 7)
summary(model, input_size=(32, 10), dtypes=[torch.long], verbose=2, device=device, col_names=["input_size", "output_size", "num_params", "mult_adds"])
model.to(device)

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Mult-Adds
NGramLanguageModel                       [32, 10]                  [32, 10, 66]              --                        --
├─Embedding: 1-1                         [32, 10, 7]               [32, 10, 7, 54]           3,564                     114,048
│    └─weight                                                                                └─3,564
├─Linear: 1-2                            [32, 10, 378]             [32, 10, 136]             51,544                    1,649,408
│    └─weight                                                                                ├─51,408
│    └─bias                                                                                  └─136
├─ReLU: 1-3                              [32, 10, 136]             [32, 10, 136]             --                        --
├─Dropout: 1-4                           [32, 10, 136]             [3

NGramLanguageModel(
  (token_embedding): Embedding(66, 54, padding_idx=0)
  (fc): Linear(in_features=378, out_features=136, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (relu): ReLU()
  (final): Linear(in_features=136, out_features=66, bias=True)
)

In [111]:
print(model)
# logits, loss = model(xb, yb)
logits = model(xb)
loss = model.loss(logits, yb)
print('logits:', logits.shape)
print('loss:', loss)

print(decode(model.generate(xb, 10)[0].tolist()))

NGramLanguageModel(
  (token_embedding): Embedding(66, 54, padding_idx=0)
  (fc): Linear(in_features=378, out_features=136, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (relu): ReLU()
  (final): Linear(in_features=136, out_features=66, bias=True)
)
logits: torch.Size([256, 10, 66])
loss: tensor(4.2138, device='cuda:0', grad_fn=<NllLossBackward0>)
wear she b'xkfE-z?FT


In [112]:
# train the model
optimizer = optim.AdamW(model.parameters(), lr=0.004)
batch_size = 256
for step in range(1001):
    xb, yb = get_batch('train')
    # logits, loss = model(xb, yb)
    logits = model(xb)
    loss = model.loss(logits, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if step % 50 == 0:
        print(f'step: {step}, loss: {loss.item():.3f}')

step: 0, loss: 4.212
step: 50, loss: 2.313
step: 100, loss: 2.140
step: 150, loss: 2.026
step: 200, loss: 2.023
step: 250, loss: 1.984
step: 300, loss: 1.925
step: 350, loss: 1.977
step: 400, loss: 1.911
step: 450, loss: 1.888
step: 500, loss: 1.894
step: 550, loss: 1.915
step: 600, loss: 1.931
step: 650, loss: 1.862
step: 700, loss: 1.877
step: 750, loss: 1.834
step: 800, loss: 1.816
step: 850, loss: 1.813
step: 900, loss: 1.862
step: 950, loss: 1.849
step: 1000, loss: 1.794


In [113]:
for x in model.generate(torch.tensor([encode("LUCENT")] * 5, device=device), 200, 0.9):
    print(decode(x.tolist()))
    print('----')

LUCENTIO:
Youpon their shall kenwermanous,
Where ore nubied: and shall from thy faire,
Is foucts of Farewill to eneHy ese. Chenole Catleryou museart then mide!

KING RICHARD:
Which swilld ad ame,
That He ti
----
LUCENTIO:
he that me me, ay to sould serve onfor and much one that opost assalt offe tian shast thy for rield; mothit bas I palsader dawnery we swell:
Ay Marry for on her viccance, my bruck ow you are much:
----
LUCENTIO:
Diven agay will my can he but when for his
Tybalted repul hid feath; morkity! thy bast thee bral a ging hist me
Wherefole!
Sy that yeak an Sime.

Lord I had come the farge your kings, you, world a
----
LUCENTIO:
Wold nother. Ance, fece
Which onfell.
Thene wotion
the cher, my life: umplencetengad-forspars not forget with daboa!

Farge you with swold, in the king our gath'd hath dend to wrich it good upon m
----
LUCENTIO:
Forion: prosmine to love, the well reto have evence, my pien's is facender and unbleould seeps even ither.

PAULINA:
Which her not, But women 

In [114]:
model.generate(torch.tensor([encode("hello")], device=device), 10, 0.0001)

tensor([[47, 44, 51, 51, 54, 62, 44, 57,  2, 59, 47, 44,  2, 58, 55]],
       device='cuda:0')

In [115]:
# save model
import os, json

os.makedirs("tmp/ngram", exist_ok=True)

with open("tmp/ngram/config.json", "w") as f:
    json.dump({"chars": chars, "vocab_size": vocab_size, "n": model.n}, f)

torch.onnx.export(
    model,
    (torch.zeros(1, 10, dtype=torch.int32, device=device), True),
    "tmp/ngram/model.onnx",
    input_names=["input"],
    output_names=["output"],
    dynamic_axes={
        "input": {0: "batch", 1: "time"},
        "output": {0: "batch", 1: "time"},
    },
)

C:\Users\shiva\AppData\Local\Temp\ipykernel_20404\978116475.py:19: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if only_last:


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x864 and 378x136)